### Import library

In [1]:
%%capture
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

!pip install emoji
import emoji

!pip install sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score, precision_score

In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

True

### Memuat Dataset

In [3]:
url = "https://raw.githubusercontent.com/Meio047/Proyek-Analisis-Sentimen/refs/heads/main/bca_mobile_reviews.csv"
df = pd.read_csv(url)
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,1bbb65ec-9126-46e9-a9ed-697b73d563e3,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,bank terramah sepanjang masa,5,0,4.6.1,2025-04-08 06:18:15,Terima kasih atas ulasannya. Semoga aplikasi B...,2025-04-08 07:37:39,4.6.1
1,b4000e13-6a3b-41de-b865-4eb433d57d09,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,pulsa udah abis banyak buat verifikasi tapi ke...,1,0,NaN,2025-04-08 06:07:59,"Mohon maaf atas ketidaknyamanan Bapak/Ibu, unt...",2025-04-08 07:37:38,NaN
2,a4923423-2ea1-427d-8af3-59355a454706,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,kode otp tidak masuk masuk tolong perbaiki,1,0,NaN,2025-04-08 05:10:12,"Mohon maaf atas ketidaknyamanan Bapak/Ibu, unt...",2025-04-08 05:20:35,NaN
3,ccbf6880-a6d8-4366-ba9b-8c6e300c8ea2,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,sangat puas,5,0,NaN,2025-04-08 05:02:04,Terima kasih atas ulasannya. Semoga aplikasi B...,2025-04-08 05:06:44,NaN
4,3a5d5110-1abc-42bb-9f6d-cdbe0eda54d1,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Bagus,4,0,4.6.1,2025-04-08 04:33:18,Terima kasih atas ulasannya. Semoga aplikasi B...,2025-04-08 05:06:43,4.6.1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   reviewId              12000 non-null  object
 1   userName              12000 non-null  object
 2   userImage             12000 non-null  object
 3   content               12000 non-null  object
 4   score                 12000 non-null  int64 
 5   thumbsUpCount         12000 non-null  int64 
 6   reviewCreatedVersion  9629 non-null   object
 7   at                    12000 non-null  object
 8   replyContent          12000 non-null  object
 9   repliedAt             12000 non-null  object
 10  appVersion            9629 non-null   object
dtypes: int64(2), object(9)
memory usage: 1.0+ MB


In [5]:
df.shape

(12000, 11)

In [6]:
df.drop(['reviewId', 'userName', 'userImage', 'score', 'thumbsUpCount',
         'reviewCreatedVersion', 'at', 'replyContent', 'repliedAt', 'appVersion'],
        axis=1, inplace=True, errors='ignore')

Beberapa kolom diatas dihapus karena tidak diperlukan lagi

In [7]:
df.head()

,content
0,bank terramah sepanjang masa
1,pulsa udah abis banyak buat verifikasi tapi ke...
2,kode otp tidak masuk masuk tolong perbaiki
3,sangat puas
4,Bagus


### Preprocesing text

In [8]:
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text)
    text = re.sub(r'#[A-Za-z0-9]+', '', text)
    text = re.sub(r'RT[\s]', '', text)
    text = re.sub(r"http\S+", '', text)
    text = re.sub(r'[0-9]+', '', text)
    text = re.sub(r'[^\w\s]', '', text)

    text = text.replace('\n', ' ')
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.strip(' ')
    return text

def casefoldingText(text): # Mengubah semua karakter dalam teks menjadi huruf kecil
    text = text.lower()
    return text

def tokenizingText(text): # Memecah atau membagi string, teks menjadi daftar token
    text = word_tokenize(text)
    return text

def filteringText(text): # Menghapus stopwords dalam teks
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    listStopwords.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"])
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text

def stemmingText(text): # Mengurangi kata ke bentuk dasarnya yang menghilangkan imbuhan awalan dan akhiran atau ke akar kata
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    stemmed_words = [stemmer.stem(word) for word in text]

    stemmed_text = ' '.join(stemmed_words)

    return stemmed_text

def toSentence(list_words): # Mengubah daftar kata menjadi kalimat
    sentence = ' '.join(word for word in list_words)
    return sentence

In [9]:
import requests
url = 'https://raw.githubusercontent.com/louisowen6/NLP_bahasa_resources/master/combined_slang_words.txt'
# Fungsi untuk mengunduh dan memproses slang words
def download_slangwords(url):
    response = requests.get(url)
    response.raise_for_status()  
    lines = response.text.splitlines()
    slang_dict = {}
    for line in lines:
        parts = line.split()
        if len(parts) == 2:
            slang_dict[parts[0].lower()] = parts[1]
    return slang_dict

slangwords = download_slangwords(url)

def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = ' '.join(fixed_words)
    return fixed_text

In [10]:
# Membersihkan teks dan menyimpannya di kolom 'text_clean'
df['text_clean'] = df['content'].apply(cleaningText)
# Mengubah huruf dalam teks menjadi huruf kecil dan menyimpannya di 'text_casefoldingText'
df['text_casefoldingText'] = df['text_clean'].apply(casefoldingText)
# Mengganti kata-kata slang dengan kata-kata standar dan menyimpannya di 'text_slangwords'
df['text_slangwords'] = df['text_casefoldingText'].apply(fix_slangwords)
# Memecah teks menjadi token (kata-kata) dan menyimpannya di 'text_tokenizingText'
df['text_tokenizingText'] = df['text_slangwords'].apply(tokenizingText)
# Menghapus kata-kata stop (kata-kata umum) dan menyimpannya di 'text_stopword'
df['text_stopword'] = df['text_tokenizingText'].apply(filteringText)
# Menggabungkan token-token menjadi kalimat dan menyimpannya di 'text_akhir'
df['text_akhir'] = df['text_stopword'].apply(toSentence)

df.head()

,content,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir
0,bank terramah sepanjang masa,bank terramah sepanjang masa,bank terramah sepanjang masa,bank terramah sepanjang masa,"[bank, terramah, sepanjang, masa]","[bank, terramah]",bank terramah
1,pulsa udah abis banyak buat verifikasi tapi ke...,pulsa udah abis banyak buat verifikasi tapi ke...,pulsa udah abis banyak buat verifikasi tapi ke...,pulsa udah abis banyak buat verifikasi tapi ke...,"[pulsa, udah, abis, banyak, buat, verifikasi, ...","[pulsa, udah, abis, verifikasi, gabisa, pesana...",pulsa udah abis verifikasi gabisa pesana terba...
2,kode otp tidak masuk masuk tolong perbaiki,kode otp tidak masuk masuk tolong perbaiki,kode otp tidak masuk masuk tolong perbaiki,kode otp tidak masuk masuk tolong perbaiki,"[kode, otp, tidak, masuk, masuk, tolong, perba...","[kode, otp, masuk, masuk, tolong, perbaiki]",kode otp masuk masuk tolong perbaiki
3,sangat puas,sangat puas,sangat puas,sangat puas,"[sangat, puas]",[puas],puas
4,Bagus,Bagus,bagus,bagus,[bagus],[bagus],bagus


### Pelabelan

In [11]:
import csv
import requests
from io import StringIO

# Membaca data kamus kata-kata positif dari GitHub
lexicon_positive = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_positive[row[0]] = int(row[1])
        # Menambahkan kata-kata positif dan skornya ke dalam kamus lexicon_positive
else:
    print("Failed to fetch positive lexicon data")

# Membaca data kamus kata-kata negatif dari GitHub
lexicon_negative = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_negative[row[0]] = int(row[1])
        # Menambahkan kata-kata negatif dan skornya dalam kamus lexicon_negative
else:
    print("Failed to fetch negative lexicon data")


In [12]:
def sentiment_analysis_lexicon_indonesia(text):
    score = 0

    for word in text:

        if (word in lexicon_positive):
            score = score + lexicon_positive[word]

    for word in text:

        if (word in lexicon_negative):
            score = score + lexicon_negative[word]

    polarity=''
    if score > 0:
        polarity = 'positive'
    elif score < 0:
        polarity = 'negative'
    else:
        polarity = 'neutral'

    return score, polarity

In [13]:
results = df['text_stopword'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
df['polarity_score'] = results[0]
df['polarity'] = results[1]
print(df['polarity'].value_counts())

polarity
negative    5459
positive    4145
neutral     2396
Name: count, dtype: int64


### Pembuatan model

Tahapan
1. Word2Vec Embedding -> Melatih representasi kata dari data sendiri
2. Tokenisasi + Padding -> Mengubah teks menjadi format numerik untuk input neural network
3. Model LSTM ->Menggunakan arsitektur Bi-LSTM untuk menangkap konteks kata
4. Hyperparameter Tuning -> Meningkatkan performa model dengan pencarian parameter optimal

In [14]:
!pip install gensim==4.3.3

In [36]:
import pandas as pd
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec
# Ambil data teks
texts = df['text_akhir'].tolist()

def tokenize_text(text):
    return text.split()  # Mengembalikan daftar token (kata)

# Tokenisasi teks
tokenized_texts = [tokenize_text(text) for text in df['text_akhir']]

# Latih model Word2Vec
model = Word2Vec(vector_size=300, window=5, min_count=3, workers=4)

# Bangun vocabulary
model.build_vocab(tokenized_texts)

print(f"Vocabulary size: {len(model.wv)}")

# Latih model
model.train(tokenized_texts, total_examples=model.corpus_count, epochs=10)

# Simpan model
model.save("word2vec_model.model")

Vocabulary size: 2437


In [37]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

model = Word2Vec.load("word2vec_model.model")

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text_akhir'])
sequences = tokenizer.texts_to_sequences(df['text_akhir'])
word_index = tokenizer.word_index

# Buat matriks embedding
vocab_size = len(word_index) + 1
embedding_dim = model.vector_size  
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in word_index.items():
    if word in model.wv:
        embedding_matrix[i] = model.wv[word]

print("Embedding matrix sample:")
print(embedding_matrix[:5])

from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

X_sequences = tokenizer.texts_to_sequences(df['text_akhir'])
max_length = max(len(seq) for seq in X_sequences)
X_padded = pad_sequences(X_sequences, maxlen=max_length)

label_encoder = LabelEncoder()

y_encoded = label_encoder.fit_transform(df['polarity'])

# Memecah data
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_encoded, test_size=0.2, random_state=42)

Embedding matrix sample:
[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.13103679  0.46979797  0.22473095 ... -0.10581052  0.58515042
   0.08734687]
 [-0.07539558  0.23952591 -0.02895861 ... -0.19566475  0.30429947
  -0.18140358]
 [-0.19879891  0.09817228 -0.25772256 ... -0.19310902  0.19679204
  -0.31252772]
 [-0.31022692 -0.34537435 -0.50013959 ... -0.17876725 -0.02901619
  -0.23413682]]


In [38]:
!pip install keras-tuner --quiet

In [39]:
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# Konversi label ke one-hot encoding
num_classes = len(label_encoder.classes_)
y_train_cat = to_categorical(y_train, num_classes=num_classes)
y_test_cat = to_categorical(y_test, num_classes=num_classes)

# Function untuk membangun model
def build_model(hp):
    model = Sequential()
    model.add(Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        weights=[embedding_matrix],
        input_length=max_length,
        trainable=hp.Boolean("trainable_embedding")
    ))
    model.add(Bidirectional(LSTM(
        units=hp.Int('lstm_units', min_value=32, max_value=128, step=32),
        return_sequences=False
    )))
    model.add(Dropout(hp.Float('dropout_rate', 0.2, 0.6, step=0.1)))
    model.add(Dense(
        units=hp.Int('dense_units', 32, 128, step=32),
        activation='relu'
    ))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(
        optimizer=Adam(learning_rate=hp.Choice('learning_rate', [1e-3, 5e-4, 1e-4])),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Inisialisasi tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='my_tuner_dir',
    project_name='word2vec_lstm_tuning'
)

# Callback
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Mulai tuning
tuner.search(X_train, y_train_cat,
             epochs=20,
             validation_split=0.2,
             callbacks=[early_stop],
             verbose=1)

# Ambil model terbaik
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluasi model terbaik
loss, accuracy = best_model.evaluate(X_test, y_test_cat)
print(f"Test Accuracy: {accuracy:.4f}")


Trial 10 Complete [00h 01m 29s]
val_accuracy: 0.7833333611488342

Best val_accuracy So Far: 0.9083333611488342
Total elapsed time: 00h 18m 04s


C:\Users\Asus\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 24 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9043 - loss: 0.3147
Test Accuracy: 0.9117


In [40]:
label_encoder.classes_

array(['negative', 'neutral', 'positive'], dtype=object)

In [47]:
texts_to_predict = [
    "Produk ini biasa saja", 
    "Sangat memuaskan, saya suka sekali!"
]

In [48]:
seqs = tokenizer.texts_to_sequences(texts_to_predict)
padded_seqs = pad_sequences(seqs, maxlen=max_length)

In [49]:
pred_probs = best_model.predict(padded_seqs)

pred_classes = pred_probs.argmax(axis=1)

# Konversi ke label kategori
predicted_labels = label_encoder.inverse_transform(pred_classes)

# Tampilkan hasil
for text, label in zip(texts_to_predict, predicted_labels):
    print(f"Teks: \"{text}\" → Prediksi: {label}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 330ms/step
Teks: "Produk ini biasa saja" → Prediksi: neutral
Teks: "Sangat memuaskan, saya suka sekali!" → Prediksi: positive
